In [796]:
import pymongo
import numpy as np
import pandas as pd
import datetime,time

In [21]:
# Connect with mongodb

try:
    with open("credentials.txt", 'r', encoding='utf-8') as f:
        [name,password,url,dbname]=f.read().splitlines()
        conn=pymongo.MongoClient("mongodb://{}:{}@{}/{}".format(name,password,url,dbname))
    print ("Connected successfully!!!")
except pymongo.errors.ConnectionFailure as e:
    print ("Could not connect to MongoDB: %s" % e) 
    

    
db = conn['tweempact']
collection = db['tweets']

Connected successfully!!!


In [22]:
db.collection_names()

C:\Users\Albert Prat\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  """Entry point for launching an IPython kernel.


['system.indexes', 'some-tweets', 'tweets', 'tweet_users']

In [23]:
def read_mongo(db, collection, query={}, host='localhost', port=27017, username=None, password=None, no_id=True):
    """ Read from Mongo and Store into DataFrame """
    
    # Make a query to the specific DB and Collection
    cursor = db[collection].find(query)

    # Expand the cursor and construct the DataFrame
    df =  pd.DataFrame(list(cursor))

    # Delete the _id
    if no_id:
        del df['_id']
    return df

In [24]:
df = read_mongo(db, "tweets")

In [25]:
len(df)

10490

In [26]:
 def author_dataset_conversion(df):   
    import time
    import numpy as np
    import pandas as pd

    cleaned = df[["id", "created_at","entities", "extended_entities","favorite_count","retweet_count","source","text", "truncated","user","quoted_status_id"]]
    ids = [user["id"] for user in cleaned["user"]]
    cleaned["id_num"] = ids

    #preparing output variables
    output = pd.DataFrame()
    users = []
    mean_RT_last10,mean_FC_last10 = [],[]
    sd_RT,sd_FC  = [],[]
    y1,y2=[],[]

    for _id in np.unique(ids):
        subset = cleaned[cleaned["id_num"]==_id].reset_index(drop=True)

        #filtering tweets from all tweets and RT
        l = list(subset["text"])
        filtr = ["RT" not in i for i in l]
        subset = subset[pd.Series(filtr)]
        if len(subset)>=11:
            reindexed = subset["created_at"].reset_index(drop=True)
            #convert tweet date into python date
            subset["created_at_pydate"] = [time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(reindexed[i],'%a %b %d %H:%M:%S +0000 %Y'))for i in range(0,len(reindexed))]
            subset = subset.sort_values(by="created_at_pydate").reset_index(drop=True)

            last10_tweets = subset[-11:-1]
            last_tweet = subset[-1:]
            #RT parameters last 10 tweets
            mean_RT_last10.append(np.mean(last10_tweets["retweet_count"]))
            sd_RT.append(np.std(last10_tweets["retweet_count"]))
            #FC parameters last 10 tweets
            mean_FC_last10.append(np.mean(last10_tweets["favorite_count"]))
            sd_FC.append(np.std(last10_tweets["favorite_count"]))
            #take json user
            j_user = subset[-1:]["user"].reset_index(drop=True)[0]
            users.append(j_user)
            #parameters of last tweet
            y1.append(np.mean(last_tweet["favorite_count"]))
            y2.append(np.mean(last_tweet["retweet_count"]))
    output["j_user"]=users
    output["RT_l10"]=mean_RT_last10
    output["sd_RT"]=sd_RT
    output["FC_l10"]=mean_FC_last10
    output["sd_FC"]=sd_FC
    output["FC"]=y1
    output["RT"]=y2
    return output

In [27]:
output = author_dataset_conversion(df)

C:\Users\Albert Prat\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [28]:
output

,j_user,RT_l10,sd_RT,FC_l10,sd_FC,FC,RT
0,"{'id': 2107671, 'id_str': '2107671', 'name': '...",0.4,0.489898,1.7,3.796051,0.0,0.0
1,"{'id': 4869021, 'id_str': '4869021', 'name': '...",8.4,16.805951,76.1,127.668673,25.0,3.0
2,"{'id': 14206520, 'id_str': '14206520', 'name':...",0.2,0.400000,2.5,3.442383,0.0,0.0
3,"{'id': 22317044, 'id_str': '22317044', 'name':...",0.0,0.000000,0.3,0.458258,0.0,0.0
4,"{'id': 22935720, 'id_str': '22935720', 'name':...",3.6,6.711185,2.8,5.015974,0.0,1.0
5,"{'id': 32406749, 'id_str': '32406749', 'name':...",0.1,0.300000,0.7,0.781025,0.0,0.0
6,"{'id': 42402745, 'id_str': '42402745', 'name':...",0.1,0.300000,0.5,0.670820,0.0,0.0
7,"{'id': 49673609, 'id_str': '49673609', 'name':...",0.8,1.249000,4.5,6.917369,1.0,0.0
8,"{'id': 56585060, 'id_str': '56585060', 'name':...",0.0,0.000000,0.0,0.000000,0.0,0.0
9,"{'id': 58451570, 'id_str': '58451570', 'name':...",1.1,1.044031,1.1,1.135782,4.0,3.0


In [26]:
pd.DataFrame.to_csv(output, "C:/Users/Albert Prat/Documents/tweempact/datasets/author.csv")

In [71]:
df["user"][0]

{'id': 2664945430,
 'id_str': '2664945430',
 'name': 'Fútbol Club Jumilla',
 'screen_name': 'FCJumilla',
 'location': 'Jumilla, España',
 'description': 'Perfil Oficial del Fútbol Club Jumilla.\nSegunda División B / Grupo IV @RFEF',
 'url': 'https://t.co/VYPxc3mFj6',
 'entities': {'url': {'urls': [{'url': 'https://t.co/VYPxc3mFj6',
     'expanded_url': 'http://www.futbolclubjumilla.es',
     'display_url': 'futbolclubjumilla.es',
     'indices': [0, 23]}]},
  'description': {'urls': []}},
 'protected': False,
 'followers_count': 7162,
 'friends_count': 334,
 'listed_count': 92,
 'created_at': 'Wed Jul 02 19:09:45 +0000 2014',
 'favourites_count': 2804,
 'utc_offset': None,
 'time_zone': None,
 'geo_enabled': True,
 'verified': False,
 'statuses_count': 8466,
 'lang': 'es',
 'contributors_enabled': False,
 'is_translator': False,
 'is_translation_enabled': False,
 'profile_background_color': 'C0DEED',
 'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme1/bg.png',
 

In [ ]:
df["user"]

In [72]:
def text_dataset(df):
    import re
    import pandas as pd
    l = list(df["text"])
    filtr = ["RT" not in i for i in l]
    df = df[filtr].reset_index(drop=True)
    
    Tweet =[re.sub("[(\n)(\r)]"," ",tweet) for tweet in df["text"]]
    entities = [tweet for tweet in df["entities"]]
    FC = [tweet for tweet in df["favorite_count"]]
    RT = [tweet for tweet in df["retweet_count"]]
    id_ = [tweet for tweet in df["id"]]
    j_user = [tweet for tweet in df["user"]]
    followers = [tweet["followers_count"] for tweet in output2["j_user"]]
    
    CA = df["created_at"]
    CA_l = [time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(CA[i],'%a %b %d %H:%M:%S +0000 %Y'))for i in range(0,len(CA))]
    
    output = pd.DataFrame([id_,Tweet,j_user,CA_l,entities,followers, FC,RT]).T
    output.reset_index(drop=True)
    output.columns = ["id", "Tweet", "j_user","created_at", "entities", "followers_user", "FC","RT"]
    return output

In [73]:
output2 = text_dataset(df)

In [76]:
pd.DataFrame.to_csv(output2, "C:/Users/Albert Prat/Documents/tweempact/datasets/tweet.csv")

In [82]:
output2.head()

,id,Tweet,j_user,created_at,entities,followers_user,FC,RT
0,1046429145914384386,Desde el #FCJumilla damos nuestra más sincera ...,"{'id': 2664945430, 'id_str': '2664945430', 'na...",2018-09-30 15:58:44,"{'hashtags': [{'text': 'FCJumilla', 'indices':...",7162,18,4
1,1044060298443919367,Me estoy volviendo loca hasta por cada una de ...,"{'id': 713359736, 'id_str': '713359736', 'name...",2018-09-24 03:05:47,"{'hashtags': [], 'symbols': [], 'user_mentions...",531,0,0
2,1046424001214648320,🏟 | El #FCJumilla ya está en La Condomina ⚽️...,"{'id': 2664945430, 'id_str': '2664945430', 'na...",2018-09-30 15:38:18,"{'hashtags': [{'text': 'FCJumilla', 'indices':...",7162,4,2
3,1046419267279818752,📹 | El vestuario del #FCJumilla en La Condomin...,"{'id': 2664945430, 'id_str': '2664945430', 'na...",2018-09-30 15:19:29,"{'hashtags': [{'text': 'FCJumilla', 'indices':...",7162,21,6
4,1046417005451128832,📑 Jornada 6 | El colegiado andaluz Guzmán Mans...,"{'id': 2664945430, 'id_str': '2664945430', 'na...",2018-09-30 15:10:30,"{'hashtags': [{'text': 'UCAMMurciaJumilla', 'i...",7162,0,0


In [84]:
hashtags = [tweet["hashtags"] for tweet in output2["entities"]]

In [101]:
hashtags[2][0]

{'text': 'FCJumilla', 'indices': [7, 17]}

In [102]:
out = []
for person in hashtags:
    l = []
    for hashtag in person:
        l.append(hashtag["text"])
    out.append(l)

In [402]:
from selenium import webdriver
browser = webdriver.Chrome(r"c:\Users\Albert Prat\chromedriver.exe")
browser.get("https://trendogate.com/")
browser.find_element_by_xpath("//select[@name='place']/option[text()='Spain']").click()
a = browser.find_element_by_xpath("//*[@id='userheaderbox']/div/form[2]/div[2]/input")
a.send_keys("01/04/2017")
browser.find_element_by_xpath("//*[@id='userheaderbox']/div/form[2]/button").click()

In [129]:
browser.find_element_by_xpath("//select[@name='place']/option[text()='Spain']").click()

In [412]:
a = browser.find_element_by_xpath("//*[@id='userheaderbox']/div/form[2]/div[2]/input")

In [414]:
a.send_keys("01/04/2017")

In [ ]:
browser.find_element_by_xpath("//*[@id='userheaderbox']/div/form[2]/button").click()

In [143]:
b = browser.find_elements_by_class_name("list-group-item")
TT = [el.text for el in b]  

In [151]:
TT = [el.text for el in b]  

In [153]:
import re

In [160]:
f = output2["Tweet"][0]

In [169]:
f

'Desde el #FCJumilla damos nuestra más sincera enhorabuena a dos enormes deportistas de la Región   🥇@alejanvalverde… https://t.co/XqLolr5sfl'

In [222]:
output=[]
for tweet in output2["Tweet"]:
    lc = [bool(re.search(word, tweet)) for word in TT]
    #lc = ["" in tweet for word in TT]
    output.append(np.sum(lc))

In [223]:
output

[0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,


In [551]:
from selenium import webdriver
browser = webdriver.Chrome(r"c:\Users\Albert Prat\chromedriver.exe")
browser.get("https://trendogate.com/")
browser.find_element_by_xpath("//select[@name='place']/option[text()='Spain']").click()
a = browser.find_element_by_xpath("//*[@id='userheaderbox']/div/form[2]/div[2]/input")


browser.find_element_by_xpath("//*[@id='userheaderbox']/div/form[2]/button").click()

In [799]:
uniq_dates

array(['2013-08-14', '2014-11-04', '2015-05-12', '2015-05-18',
       '2015-05-24', '2015-05-31', '2015-06-08', '2015-06-13',
       '2015-08-03', '2015-08-08', '2015-08-09', '2015-09-29',
       '2015-09-30', '2015-10-08', '2015-10-12', '2016-01-03',
       '2016-01-06', '2016-02-02', '2016-02-03', '2016-02-04',
       '2016-02-05', '2016-02-06', '2016-03-05', '2016-03-29',
       '2016-04-16', '2016-05-03', '2016-05-07', '2017-02-23',
       '2017-05-21', '2017-06-03', '2017-06-04', '2017-07-23',
       '2017-08-04', '2017-10-02', '2017-11-27', '2017-12-16',
       '2017-12-20', '2017-12-23', '2018-03-10', '2018-03-30',
       '2018-04-02', '2018-04-04', '2018-04-05', '2018-04-07',
       '2018-04-23', '2018-04-24', '2018-04-25', '2018-04-27',
       '2018-05-15', '2018-05-19', '2018-05-21', '2018-05-24',
       '2018-05-25', '2018-05-26', '2018-05-27', '2018-05-28',
       '2018-05-30', '2018-05-31', '2018-06-01', '2018-06-02',
       '2018-06-04', '2018-06-05', '2018-06-06', '2018-

In [802]:
u = [datetime.datetime.strptime(date, "%Y-%m-%d") for date in uniq_dates]
threshold = datetime.datetime.strptime("01-03-2015", "%d-%m-%Y")

In [808]:
filtr = [u[i]>=threshold for i in range(len(u))]

In [813]:
len(uniq_dates)

182

In [818]:
from selenium import webdriver
browser = webdriver.Chrome(r"c:\Users\Albert Prat\chromedriver.exe")
browser.get("https://trendogate.com/")
browser.find_element_by_xpath("//select[@name='place']/option[text()='Spain']").click()
textdf = output2["Tweet"]
cadf = output2["created_at"]
cadf = [date[:10] for date in cadf]
uniq_dates = np.unique(cadf)
uniq_dates = uniq_dates[filtr]

error = []
grepl = []
out = []
date_dd = browser.find_element_by_xpath("//*[@id='userheaderbox']/div/form[2]/div[2]/input")
for i in range(len(uniq_dates)):
    browser.find_element_by_xpath("//select[@name='place']/option[text()='Spain']").click()
    date_dd = browser.find_element_by_xpath("//*[@id='userheaderbox']/div/form[2]/div[2]/input")
    date = uniq_dates[i][:10]
    date = date.split("-")
    date = date[2]+date[1]+date[0]
    date_dd.send_keys(date)
    browser.find_element_by_xpath("//*[@id='userheaderbox']/div/form[2]/button").click()
    
    if len(browser.find_elements_by_class_name("alert")) == 1:
        error.append(date)
        out.append(None)
        pass
    else:
        b = browser.find_element_by_class_name("col-lg-8")
        lgi = b.find_elements_by_class_name("list-group-item")
        TT = [el.text for el in lgi]
        out.append(TT)

In [895]:
l = [None]
out = l+l+out

In [888]:
error

['18052015', '24052015', '03012016', '06012016']

In [897]:
uni = list(np.unique(cadf))

In [898]:
a = [date.split("-") for date in uni]

In [900]:
a = [date[2]+date[1]+date[0] for date in a]

In [902]:
index = [a.index(e) for e in error]

In [584]:
for i in index:
    del a[i]

In [ ]:
[",".join(tt) for tt in out]

In [903]:
tocsv = pd.DataFrame([a, out]).T

In [907]:
tocsv.head(7)

,0,1
0,14082013,None
1,04112014,None
2,12052015,"[The Division, Paco Ureña, #LoMas40porLaTarde,..."
3,18052015,None
4,24052015,None
5,31052015,"[Enrique Iglesias, Higuaín, Nápoles, Bienvenid..."
6,08062015,"[La UDEF, Stannis, Kate Moss, Paco Bustos, Dan..."


In [911]:
tocsv.columns = ["day", "trending_topic"]

In [912]:
tocsv[tocsv.index == 150]

,day,trending_topic
150,25092018,"[#CocinasTezanos, #DMF2018, #AnimalesFantástic..."


In [ ]:
for tweet in textdf:
    ls = []
    tweet = tweet.lower()
    for word in TT:
        ls.append(bool(re.search(r"\b"+str(word)+r"\b", tweet)))
grepl.append(np.sum(ls))

In [913]:
tocsv.to_csv(r"C:\Users\Albert Prat\Documents\tweempact\datasets\trending_topics.csv")

In [610]:
sp_d = [line[:10] for line in output2["created_at"]]

In [612]:
sp_d = []
for line in output2["created_at"]:
    l = line[:10]
    l = l.split("-")
    l = l[2]+l[1]+l[0]
    sp_d.append(l)

['30092018',
 '24092018',
 '30092018',
 '30092018',
 '30092018',
 '30092018',
 '30092018',
 '30092018',
 '30092018',
 '27092018',
 '30092018',
 '29092018',
 '29092018',
 '29092018',
 '28092018',
 '28092018',
 '28092018',
 '28092018',
 '28092018',
 '27092018',
 '27092018',
 '30092018',
 '27092018',
 '26092018',
 '26092018',
 '25092018',
 '25092018',
 '22092018',
 '16092018',
 '16092018',
 '16092018',
 '16092018',
 '16092018',
 '16092018',
 '28092018',
 '30092018',
 '29092018',
 '27092018',
 '27092018',
 '27092018',
 '27092018',
 '26092018',
 '26092018',
 '26092018',
 '26092018',
 '30092018',
 '29092018',
 '29092018',
 '29092018',
 '29092018',
 '29092018',
 '27092018',
 '27092018',
 '27092018',
 '27092018',
 '27092018',
 '26092018',
 '26092018',
 '26092018',
 '26092018',
 '26092018',
 '25092018',
 '25092018',
 '25092018',
 '30092018',
 '30092018',
 '30092018',
 '30092018',
 '30092018',
 '30092018',
 '29092018',
 '29092018',
 '28092018',
 '28092018',
 '28092018',
 '27092018',
 '27092018',

In [614]:
index = [sp_d.index(e) for e in er]

In [615]:
index

[2875, 2874, 2862, 2861]

In [623]:
output2[output2.index == 2875]

,id,Tweet,j_user,created_at,entities,followers_user,FC,RT
2875,600443570433105920,@TownGamePlay puedes mandarle saludos a mi he...,"{'id': 2889180230, 'id_str': '2889180230', 'na...",2015-05-18 23:31:17,"{'hashtags': [], 'symbols': [], 'user_mentions...",10,0,0


In [638]:
textdf = output2["Tweet"]
# date management
ca = output2["created_at"]
sp_d = []
for date in ca:
    ca_tweet = date[:10]
    ca_tweet = ca_tweet.split("-")
    ca_tweet = ca_tweet[2]+ca_tweet[1]+ca_tweet[0]
    sp_d.append(ca_tweet)

In [641]:
textdf = output2["Tweet"]
# date management
ca = output2["created_at"]
sp_d = []
for date in ca:
    ca_tweet = date[:10]
    ca_tweet = ca_tweet.split("-")
    ca_tweet = ca_tweet[2]+ca_tweet[1]+ca_tweet[0]
    sp_d.append(ca_tweet)
unique_dates = np.unique(sp_d)

In [958]:
textdf = output2["Tweet"]
# date management
ca = output2["created_at"]
unique_date = list(np.unique(ca))
unique_date = [date[:10] for date in unique_date]
unique_date = np.unique(unique_date)
unique_date = [date.split("-") for date in unique_date]
unique_date = [date[2]+date[1]+date[0] for date in unique_date]

sp_d = []
for date in ca:
    ca_tweet = date[:10]
    ca_tweet = ca_tweet.split("-")
    ca_tweet = ca_tweet[2]+ca_tweet[1]+ca_tweet[0]
    sp_d.append(ca_tweet)

    

output2["spanish_date"]=sp_d
grepl = []
for i in range(len(sp_d)):
    ls = []
    date = output2[output2.index == i]["spanish_date"][i]
    tweet = textdf[i].lower()
    index = unique_date.index(date)
    TT = out[index]
    if TT == None:
        grepl.append(None)
    else:
        for word in TT:
            word = word.lower()
            ls.append(bool(re.search(r"\b"+str(word)+r"\b", tweet)))
        grepl.append(np.sum(ls))

In [959]:
sum(filter(None, grepl))

25

In [943]:
np.where(np.array(grepl) == 1)

(array([  66,  509,  556,  563,  639,  640,  826, 1129, 1145, 1179, 1189,
        1392, 1393, 1398, 1771, 1783, 1990, 2207, 2209, 2261, 2737, 2927,
        2955, 3319, 3685], dtype=int64),)

In [962]:
output2["trending_topic"] = grepl

In [969]:
output2 = output2.drop("spanish_date", axis = 1)

In [970]:
output2.to_csv(r"C:\Users\Albert Prat\Documents\tweempact\datasets\tweet.csv")